<a target="_blank" href="https://colab.research.google.com/github/kavyajeetbora/ETL_wages/blob/master/ETL_wages_data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# @title Setup the environment and Upload files

# @markdown Run this only once otherwise restart the session
!pip install aspose-cells
import  jpype
jpype.startJVM()
from asposecells.api import Workbook
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
import os
from google.colab import files

def convert_xl2xlsx(filename):
    '''
    Convert xls file to xlsx
    '''
    if os.path.isfile(filename):
        workbook = Workbook(filename)
        export_filename = f"{os.path.basename(filename)}.xlsx"

        if os.path.isfile(export_filename):
            os.remove(export_filename)

        workbook.save(export_filename)
        return export_filename
    else:
        return None

def preprocess_data(xlsx_file):
    '''
    Preprocessing the dataframe:
    - setting the row id for table header
    - Setting the year column as table index
    - Limiting the table to the last year, current 18th row number
    '''
    df = pd.read_excel(xlsx_file, header=[4,5], nrows=18)
    df = df.drop(index=[0,1],axis=0).reset_index(drop=True)
    df = df.set_index(df.columns[0])
    df.index.names = ['Year']
    # df.tail()
    return df

def unpivot_table(df):
    '''
    Unpivot the table

    '''
    df_out = {'year':[],'sector':[]}
    sectors = []
    years = []

    for cols in df.columns:
        i1,i2 = map(str.strip, cols)
        values = list(df.loc[:,cols].values)

        if i1 not in df_out.keys():
            df_out.setdefault(i1,values)
        else:
            df_out[i1].extend(values)

        i2 = i2.split("-")[0].strip()
        if i2 not in sectors:
            df_out['sector'].extend([i2]*len(values))
            df_out['year'].extend(df.index)
        sectors.append(i2)

    df_out = pd.DataFrame(df_out)
    return df_out

def post_process_df(df):
    df_out = df.copy()
    df_out = df_out.replace(r'^\s*$', float('nan'), regex=True)
    for col in df_out.columns[2:]:
        df_out[col] = df_out[col].astype(float)
    df_out['sector']=df_out['sector'].astype(int)
    return df_out

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 28.0 MB/s eta 0:00:00
  Created wheel for aspose-cells: filename=aspose_cells-24.3.0-py3-none-any.whl size=14226572 sha256=229434df03fda22149f091f0f8a12fed2ff685d933f978a94d432c9e8e7310f0
  Stored in directory: /root/.cache/pip/wheels/bf/10/47/f3d06fe998b7fc334dbd7561c7ba6204e8ae612a35d7850cb4
Successfully built aspose-cells


In [2]:
# @title 1. Upload files
## Clear all existing files
_ = [os.remove(x) for x in os.listdir() if os.path.isfile(x)]

# Prompt user to upload a folder
uploaded = files.upload()

Saving 200_224_wages_1970.xls to 200_224_wages_1970.xls
Saving 250_274_wages.xls to 250_274_wages.xls
Saving 275_299_wages.xls to 275_299_wages.xls
Saving 300_324_wages.xls to 300_324_wages.xls
Saving 325_349_wages.xls to 325_349_wages.xls
Saving 350_374_wages.xls to 350_374_wages.xls
Saving 375_rest_wages.xls to 375_rest_wages.xls


In [3]:
# @title 2. Process the data
## ETL

xl_files = glob("*.xls")

dfs = []
for xl_file in tqdm(xl_files, unit=' xl file'):
    try:
        xlsx_file = convert_xl2xlsx(xl_file)

        df = preprocess_data(xlsx_file)
        df_out = unpivot_table(df)
        df_out=post_process_df(df_out)
        dfs.append(df_out)
    except Exception as e:
        print(xl_file,e)

  0%|          | 0/7 [00:00<?, ? xl file/s]

200_224_wages_1970.xls header index 4 exceeds maximum index 0 of data.


In [4]:
# @title 3. Display Output
df_final = pd.concat(dfs)
df_final.sample(5)

,year,sector,Fixed Capital,Number of Workers,Wages and Salaries - Workers,Gross Value Added,Wages and Salaries - Total
278,1979 - 1980,741,2961.0,7644.0,233.0,1194.0,NaN
182,1979 - 1980,266,58.0,503.0,24.0,111.0,NaN
240,1973 - 1974,410,1508.0,2877.0,135.0,587.0,NaN
202,1983 - 1984,288,276.0,1758.0,113.0,407.0,NaN
302,1987 - 1988,273,823.0,2701.0,202.0,858.0,NaN


from matplotlib import pyplot as plt
_df_0['sector'].plot(kind='hist', bins=20, title='sector')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Fixed Capital'].plot(kind='hist', bins=20, title='Fixed Capital')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Number of Workers'].plot(kind='hist', bins=20, title='Number of Workers')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Wages and Salaries - Workers'].plot(kind='hist', bins=20, title='Wages and Salaries - Workers')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['year'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='Fixed Capital', y='year', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['year'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='Number of Workers', y='year', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['year'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='Wages and Salaries - Workers', y='year', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [5]:
# @title 4. Export the file
## Clear all existing files once again

_ = [os.remove(x) for x in os.listdir() if os.path.isfile(x)]
df_final.to_excel("output.xlsx")
files.download('output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

References

1. [Apose cells](https://products.aspose.com/cells/python-java/conversion/xls-to-xlsx/)
2. [Google forms](https://colab.research.google.com/notebooks/forms.ipynb)